In [ ]:
import pandas as pd

#### Vérification du fichier

In [ ]:
filename = "economicref-france-sirene-v3.csv"
with open(filename, "rb") as f:
    print(f.read(100))

---

#### Stats

In [ ]:
# Visualisation des données
df=pd.read_csv(filename, sep=";")
pd.set_option('display.max_columns', None)
df.head()

In [ ]:
print("Number of rows : {}".format(df.shape[0]))
print()

print("Basics statistics: ")
df_desc = df.describe(include="all")
display(df_desc)
print()

print("Total and percentage of missing values: ")
missing_total = df.isnull().sum().sum()
display(missing_total)
missing_percent = 100 * df.isnull().sum() / df.shape[0]
display(missing_percent[missing_percent != 0])

In [ ]:
df.columns.to_list()

In [ ]:
df['Nature juridique de l\'unité légale'].value_counts()

In [ ]:
a_exclure = [
    "Entrepreneur individuel",
    "Société civile immobilière",
    "Société civile immobilière de construction-vente",
    "Société étrangère non immatriculée au RCS",
    "Association déclarée",
]

df = df[~df["Nature juridique de l'unité légale"].isin(a_exclure)]

In [ ]:
pd.set_option("display.max_rows", None)
df["Nature juridique de l'unité légale"].value_counts()

In [ ]:
df.shape

In [ ]:
# SAS classique exacte
mask_sas_classique = df["Nature juridique de l'unité légale"] == "SAS, société par actions simplifiée"

# SARL exacte
mask_sarl_exacte = df["Nature juridique de l'unité légale"] == "Société à responsabilité limitée (sans autre indication)"

# Conserver uniquement ces deux formes
df_filtered = df[mask_sas_classique | mask_sarl_exacte]

# Vérification
print(df_filtered["Nature juridique de l'unité légale"].value_counts())

In [ ]:
df_filtered.shape

In [ ]:
df_filtered.head()

---

In [ ]:
df_filtered.dtypes

##### Extraction du dataset en Parquet

In [ ]:
# 1. On crée une copie propre pour isoler df_filtered
df_final = df_filtered.copy()

# 2. On traite les colonnes problématiques

for col in df_final.columns:
    if df_final[col].dtype == 'object':
        df_final[col] = df_final[col].fillna('').astype(str)

# 3. On force aussi les colonnes de codes (souvent en float à cause des NaN)
cols_codes = [
    "Code postal de l'établissement", 
    "Code commune de l'établissement",
    "Code du pays de l'établissement étranger",
    "Code EPCI de l'établissement"
]

for col in cols_codes:
    if col in df_final.columns:
        df_final[col] = df_final[col].fillna('').astype(str)

# 4. Exportation finale en Parquet
df_final.to_parquet('sas_sarl_filtered.parquet', index=False, engine='pyarrow')

print("Succès ! Le fichier est prêt pour l'EDA.")

In [ ]:
# df_filtered.to_csv('sas_sarl_filtered.csv', index=False, encoding="utf-8")